In [ ]:
from __future__ import absolute_import
from __future__ import print_function
import os


os.environ['KERAS_BACKEND'] = 'theano'
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=cuda0,floatX=float32,optimizer=None'


import keras
import keras.models as models

from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import BatchNormalization,Input
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers.convolutional import Convolution2D
from keras.optimizers import SGD, Adam, RMSprop
import sklearn.metrics as metrics

from keras.callbacks import ModelCheckpoint

import cv2
import numpy as np
import json

# from Dan Does Data VLOG
import math
import h5py
import glob
from tqdm import tqdm
import scipy
from scipy import misc

import matplotlib.pyplot as plt
plt.ion()


imgfiles = sorted(glob.glob('data/*-images.npz'))
steerfiles = sorted(glob.glob('data/*-steer.npz'))
# TODO: Handle more than one of each file (concat)
imgs = np.load(imgfiles[0])['img_arr']
targets = np.load(steerfiles[0])['steer_arr']

print(f'Have {imgs.shape[0]} images')

In [ ]:
# load the model:
model = Sequential()
with open('autopilot_basic_model.json') as model_file:
    model = models.model_from_json(model_file.read())


# load weights
model.load_weights("weights/model_basic_weight.hdf5")


adam = Adam(lr=0.0001)
model.compile(loss='mse',
              optimizer=adam,
              metrics=['mse','accuracy'])


preds = model.predict(imgs)

In [ ]:
preds = preds.reshape(len(preds)) * 1.5 # TODO: should not need to * 1.5
plt.figure(0, figsize=(12, 10))
plt.plot(np.array([targets[0:200], preds.reshape(len(preds))[0:200]]).T, '.')

err = preds - targets
plt.figure(1, figsize=(12, 10))
plt.plot(err, '.')